In [ ]:
# Importing Libraries
import os
import glob
import datetime
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.models import model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.regularizers import L2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, LeakyReLU, BatchNormalization, ReLU
import urllib.request
import zipfile

# Set TensorFlow version
tf.__version__

# Constants for image size, data split, and random state
IMAGE_SIZE = (128, 128)
RANDOM_STATE = 7
TRAIN_SIZE, VAL_SIZE, TEST_SIZE = 0.8, 0.1, 0.1
import os
import urllib.request
import tarfile

# Define URL and directory
DATA_URL = "https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz"
DATA_DIR = "/content/oxford_pets_data"
FILE_PATH = os.path.join(DATA_DIR, "images.tar.gz")

# Create the directory if it doesn't exist
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# Download the dataset if not already downloaded
if not os.path.exists(FILE_PATH):
    print("Downloading dataset...")
    urllib.request.urlretrieve(DATA_URL, FILE_PATH)
    print("Download complete!")

# Extract the dataset
print("Extracting dataset...")
with tarfile.open(FILE_PATH, 'r:gz') as tar_ref:
    tar_ref.extractall(DATA_DIR)
print("Dataset downloaded and extracted!")

# Verify the dataset is downloaded
BASE_PATH = os.path.join(DATA_DIR, "images")  # Adjust this if necessary
imageNames = [os.path.basename(file) for file in glob.glob(os.path.join(BASE_PATH, '*.jpg'))]
print(f"Total number of image files: {len(imageNames)}")

# Reading target label from filenames
labels = [' '.join(name.split('_')[:-1]) for name in imageNames ]
print(f"Total number of unique labels: {len(np.unique(labels))}")

# Label Encoding and lookup dictionary
labelEncDict = {name: ind for ind, name in enumerate(np.unique(labels))}
for k, v in labelEncDict.items():
    print(f"{k:32} : {v}")

# Reverse lookup dictionary
labelDecDict = {ind: name for name, ind in labelEncDict.items()}

# Loading the image data
imageData = []
for name in tqdm(imageNames, desc = 'Loading image data', unit = ' images'):
    img = load_img(os.path.join(BASE_PATH, name))
    img = tf.image.resize_with_pad(img_to_array(img, dtype = 'uint8'), *IMAGE_SIZE).numpy().astype('uint8')
    imageData.append(img)

imageData = np.array(imageData)
imageData.shape

# Encoding Target labels
labelsEncoded = list(map(lambda x : labelEncDict.get(x), labels))
for i, l in zip(imageNames[::1000], labelsEncoded[::1000]):
    print(f"{i:32}\t{labelDecDict[l]:32}\t{l}")

# Split data and labels into Train, Test, and Validation sets
X_tv, X_test, y_tv, y_test = train_test_split(
    imageData,
    labelsEncoded,
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE,
    stratify = labelsEncoded
)

X_train, X_val, y_train, y_val = train_test_split(
    X_tv,
    y_tv,
    test_size = VAL_SIZE,
    random_state = RANDOM_STATE,
    stratify = y_tv
)

print(f'Training Data: {X_train.shape}')
print(f'Training Labels: {len(y_train)}')
print(f'\nValidation Data: {X_val.shape}')
print(f'Validation Labels: {len(y_val)}')
print(f'\nTesting Data: {X_test.shape}')
print(f'Testing Labels: {len(y_test)}')

# Image Data Generator for training, validation, and test data
train_gen = ImageDataGenerator(rescale = 1./255,
                               rotation_range = 30,
                               width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               shear_range = 0.1,
                               zoom_range = 0.1,
                               horizontal_flip = True,
                               fill_mode = 'nearest')
train_data = train_gen.flow(x = X_train, y = y_train, batch_size = 32, shuffle = True)

val_gen = ImageDataGenerator(rescale = 1./255)
val_data = val_gen.flow(x = X_val, y = y_val, batch_size = 32, shuffle = True)

test_gen = ImageDataGenerator(rescale = 1./255)
test_data = test_gen.flow(x = X_test, y = y_test, batch_size = 32)

# CNN Model Creation
model = Sequential([
    Conv2D(32, 5, padding = 'same', input_shape = (*IMAGE_SIZE, 3)),
    Conv2D(32, 5, padding = 'same', activation = LeakyReLU(alpha = 0.5)),
    MaxPooling2D(),
    Conv2D(32, 4, padding = 'same'),
    Conv2D(32, 4, padding = 'same', activation = LeakyReLU(alpha = 0.5)),
    MaxPooling2D(),
    Conv2D(64, 4, padding = 'same'),
    Conv2D(64, 4, padding = 'same', activation = LeakyReLU(alpha = 0.5)),
    MaxPooling2D(),
    BatchNormalization(),
    Conv2D(64, 3, padding = 'same'),
    Conv2D(64, 3, padding = 'same', activation = LeakyReLU(alpha = 0.5)),
    MaxPooling2D(),
    Conv2D(128, 3, padding = 'same'),
    Conv2D(128, 3, padding = 'same', activation = LeakyReLU(alpha = 0.5)),
    MaxPooling2D(),
    Conv2D(128, 2, padding = 'same'),
    Conv2D(128, 2, padding = 'same', activation = LeakyReLU(alpha = 0.5)),
    MaxPooling2D(),
    Conv2D(256, 2, padding = 'same'),
    Conv2D(256, 2, padding = 'same', activation = LeakyReLU(alpha = 0.5)),
    MaxPooling2D(),
    Flatten(),
    BatchNormalization(),
    Dropout(0.2),
    Dense(512, activation = 'sigmoid'),
    Dropout(0.2),
    Dense(256, activation = 'sigmoid'),
    Dropout(0.1),
    Dense(len(labelEncDict), activation = 'softmax')
])

# Compile the model
model.compile(optimizer = Adam(learning_rate=0.0001),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['sparse_categorical_accuracy'])

model.summary()

# Train the model
history = model.fit(train_data, validation_data = val_data, epochs = 50, verbose = 1)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, verbose = 0)
print(f"Loss on Testing data: {test_loss}")
print(f"Accuracy on Testing data: {test_acc}")

# Saving the trained model
model.save('/content/oxford_pets_model')

# Load Saved Model
saved_model = tf.keras.models.load_model('/content/oxford_pets_model')

# Evaluate the loaded model
sm_test_loss, sm_test_acc = saved_model.evaluate(test_data, verbose = 0)
print(f"Loss on Testing data (loaded model): {sm_test_loss}")
print(f"Accuracy on Testing data (loaded model): {sm_test_acc}")

1. Download & Extract Dataset
Downloads the Oxford-IIIT Pet Dataset (images.tar.gz).

Extracts image files to oxford_pets_data/images/.

2. Preprocess Data
Reads image filenames and extracts class labels.

Encodes labels into numerical values.

Resizes images to (256, 256).

Splits data into Train (80%), Validation (10%), and Test (10%).

3. Data Augmentation
Uses ImageDataGenerator to apply transformations such as rotation, zoom, and flips to training data.

4. Build CNN Model
A deep Convolutional Neural Network (CNN) with:

Multiple Conv2D layers for feature extraction.

LeakyReLU activation to avoid vanishing gradients.

MaxPooling2D layers to reduce spatial dimensions.

BatchNormalization and Dropout for regularization.

Fully connected (Dense) layers with softmax activation for classification.

5. Compile & Train Model
Uses the Adam optimizer (lr=0.0001) with sparse_categorical_crossentropy loss.

Trains for 50 epochs, tracking accuracy and loss.

6. Evaluate & Save Model
Evaluates the model on test data.

Saves the trained model as oxford_pets_model.

Loads the saved model and re-evaluates its performance.

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Constants
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 50
RANDOM_STATE = 7
DATA_DIR = "./oxford_pets_data"
BASE_PATH = os.path.join(DATA_DIR, "images")

# Load images and labels
image_names = [os.path.basename(file) for file in glob.glob(os.path.join(BASE_PATH, '*.jpg'))]
labels = [' '.join(name.split('_')[:-1]) for name in image_names]
label_enc_dict = {name: ind for ind, name in enumerate(np.unique(labels))}
label_dec_dict = {ind: name for name, ind in label_enc_dict.items()}

# Process images
image_data = []
for name in tqdm(image_names, desc='Loading images'):
    img = load_img(os.path.join(BASE_PATH, name), target_size=IMAGE_SIZE)
    img = img_to_array(img) / 255.0  # Normalize
    image_data.append(img)

image_data = np.array(image_data)
labels_encoded = np.array([label_enc_dict[label] for label in labels])

# Split data
X_tv, X_test, y_tv, y_test = train_test_split(image_data, labels_encoded, test_size=0.1, stratify=labels_encoded, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.1, stratify=y_tv, random_state=RANDOM_STATE)

# Data augmentation
train_gen = ImageDataGenerator(rotation_range=30, horizontal_flip=True)
train_data = train_gen.flow(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
val_data = ImageDataGenerator().flow(X_val, y_val, batch_size=BATCH_SIZE)
test_data = ImageDataGenerator().flow(X_test, y_test, batch_size=BATCH_SIZE)

# Model using MobileNetV2 as feature extractor
base_model = MobileNetV2(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(len(label_enc_dict), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS, callbacks=[early_stopping])

# Plot training history
def plot_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].plot(history.history['accuracy'], label='Train Accuracy')
    ax[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax[0].set_title('Accuracy')
    ax[0].legend()

    ax[1].plot(history.history['loss'], label='Train Loss')
    ax[1].plot(history.history['val_loss'], label='Validation Loss')
    ax[1].set_title('Loss')
    ax[1].legend()

    plt.show()

plot_history(history)

# Evaluate model
test_loss, test_acc = model.evaluate(test_data)
print(f'Test Accuracy: {test_acc}')

# Confusion Matrix
y_pred = np.argmax(model.predict(X_test), axis=1)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(label_enc_dict.keys()))
disp.plot(figsize=(12, 12))
plt.show()

# Save model
model.save('oxford_pets_model.h5')
